# Standard rules (to be applied first): 
Note that we need to flip the capitalization in order to find the exceptions. However, all rules force capitalization, so perhaps only decapitalize and mark exceptions. Or maybe we can do everything together. 
 - Capitalization of the first letter in a sentence: Applied to first letter following ('.','!','?','\n\n') as well as first letter of text. '.. ' doesn't count, while '...' counts. The text seems inconsistent on this rule. '\n\n' replaces '\n\t' due to using internet style of paragraphs, which avoid '\t' due to it's use for document control. (I think I forgot about the first letter.)
 - Capitalization of the title: The titles here are marked as \*text\*, so the rule would be everything (except of, and, the, etc. ) until the first star, and likewise on lines with * but not ** together. Due to the difficulty of transmittion, non-standard form, and fact that there actually are not many titles besides the table of contents, I removed this rule.
 - Capitalization of 'I' when surrounded only by symbols: Note that many of these cases are triggered by the first rule, so I need to be careful about finding exceptions.
Proper nouns should be done as exceptions, as well as abbreviations.

In [181]:
#libraries
import string
import re
import numpy

In [183]:
#load text
f = open("MrZoat_WithThisRingS01_Gould.txt")
text = f.read()
f.close()
#print(text[0:200])

In [22]:
#t2 = text.casefold()
#print(t2[0:200])

In [185]:
sim = ['*', '+', '|', '/', ':', '-', ';', '\'', '\"', ',', '.', '?', '!', '$', '%', '&', '}', '[', ']', '(', ')', '<', '>', ' ', '\n']
#charlist = ['*+|/:-;\'\",.?!$%&}[]()<> \n']
len(sim)

25

In [26]:
for i in sim:
    print(text.count(i))

5793
28
1
111
522
5717
314
34222
42394
28972
77648
11159
3318
1
4
6
1
27
22
8
35
1
2
724362
57012


Test to see that the text is in ascii and that there are limited types of whitespace.

In [28]:
clist2 = ['  ', '\n ', ' \n', '\n\n\n', '\t']

In [30]:
for i in clist2:
    print(text.count(i))

0
0
0
0
0


In [32]:
text.isascii()

True

In [34]:
nl = print(text.count('\n'))
nl2 = print(text.count('\n\n'))
nl1 = 57012-2*28242
print(nl1)

57012
28242
528


In [213]:
low = []
upp = []
for i in range(26):
    low.append(string.ascii_lowercase[i])
    upp.append(string.ascii_uppercase[i])
num = []
for i in range(10):
    num.append(string.digits[i])

In [38]:
for i in low:
    print(text.lower().count(i))

260816
51386
78609
114678
373960
61620
79781
184312
234962
6536
39224
131581
88726
230955
254119
57170
2531
184912
200581
310442
100061
27934
69026
4939
67958
2833


In [42]:
for i in num:
    print(text.count(i))

310
436
322
166
129
329
135
111
79
91


In [46]:
#t1 = text

From the original text we can use a set of rules, flip the case of all which conform to that rule, and look for capitals
First rule: - titles are completely capitalized
Second rule: - the first letter of sentences are capitalized. 

In [44]:
text.find('\t')

-1

In [1134]:
#print(tout[700:1000])

In [1136]:
#print(tout[-103400:-103300])

In [1138]:
#tout = ""
#st0=0
#st1=t1[st0:].find('*')+st0
#print(st1)
#st2 = t1[st1:-1].find('\n')+st1+1
#tout += t1[0:st2].lower()#.title()
#st0 = st2 +1
#print(st0,st1,st2)
#while st0 >= 0:
#    st1=t1[st0:].find('*')
#    if st1 < 0:
#        break
#    st1 = st1 + st0
#    if t1[st1+1]!='*':
        #print(st0,st1,st2)
#        tout += t1[st2:st1]
#        st2 = t1[st1:-1].find('\n')+st1
#        tf = t1[st1:st2].lower()#.title()
#        tout +=tf
#        st0=st2
#    else:
#        st0 = st1+2
    #if st0 > 10000:
    #st0 = -1
#print(k)
#tout+=t1[st2:]
#print(len(tout)-len(t1))
#t1 = tout

Now we need a list of all our names including 'I'. 

In [1140]:
#print(t3[0:1000])

# Exceptions

The main point here is that we have a list of exceptions ...
Separating out the names doesn't seem to be the most effective. Instead we just have an array of spaces between exceptions. Marking spacings between exceptions rather than distance from the start gives the same information, but with smaller numbers. ABCD... would be A = index of first exception, B = index of second exception - (A+1) so that if B = 0, B is the next element after A. We can then choose a number encoding to reduce the size of this array. 

For a text length of > 4 mil, currently found 55500 exceptions = average spacing of 76. Number of words with exceptions is ~100 average spacing. Most likely Fibinachi is the best variant for saving space based on requiring efficient storage for all ranges to ~ 200. 

File size ~ 4 MB. 55500 * 10 = 555000 kb = ~ 70000 kB. 

In theory I could have two types of exceptions, a name list and a separate exception list. 
abbr. list;  stop; name list; stop; exception list; stop; text

In [187]:
len(re.findall('[A-Z][A-Z][a-z]', text))

32

abbreviation list
Note that with 26 options, 5 bits is enough to store each letter, with 6 stops = 3 bits at the end for the stop (18 bits each). Encoding should be simple: alphabetical order.

000xx = stop
0010x = stop
0011x = ab
01xxx = c-j
10xxx = k-r
11xxx = s-z

(1 - 16, 0 - 10 + 6 -> 1 - 8, 0 - 2 + 6 -> 1 - 2 + 2, 0 - 4)

The following were found (threshold 5): 
SCU EMP FTL GMT GBS HUD WXYZ BABUM
The following were found (threshold 3): 
SCU EMP FTL CIA BBC GMT GBS DCU HUD GED WXYZ BABUM GCSE XCOM 

In [189]:
abr = []

set01 = set(re.findall(' [A-Z][A-Z][A-Z] ', text))

for i in set01:
    if text.count(i.strip()) > 2 and text.count(i.lower()) < 2:
        abr.append(i.strip())

set02 = set(re.findall(' [A-Z][A-Z][A-Z][A-Z][A-Z.?! ]', text))

for i in set02:
    if text.count(i.strip()) > 2 and text.count(i.lower()[0:5]) < 3 and text.count(i.lower().strip().capitalize()) < 3:
        abr.append(i.strip())

abr

['EMP',
 'GED',
 'SCU',
 'CEO',
 'CIA',
 'FTL',
 'BBC',
 'GMT',
 'DCU',
 'HUD',
 'GBS',
 'XCOM',
 'WXYZ',
 'BABUM',
 'GCSE']

BABUM does appear lowercase, but more often all uppercase. MONQI is more often lowercase. BABUM is no longer found. HAHAH is new, but also often lowercase.

Name list
Names also have a standard form, so character count is also 26. The efficency of transmition depends on the length of the name however. Longer names could in principle be shortened, since longer text strings are unlikely to repeat in an alternative context. Unlike abbreviations, character frequency and context mean something, and there are many more names. However, I don't want to deal with setting up such an encoding right now, and the frequencies have lower dependence than normal words due to language mixing. 

If transmittion of an exception requires ~ 10-12 bits, while transmittion of a name requires 3 + 5 * length bits (or 8 * length bits), the count should be equal to about half the length, perhaps up to the length. Just be sure that it doesn't appear in lowercase form.

In [52]:
#print(len(set03), len(set04))

In [191]:
w3=re.findall('[A-Z][a-z]+', text)
set03 = list(set(w3))

In [739]:
#for i in range(len(set03)):
#    set03[i] = set03[i][0:4]

In [193]:
set04 = []
for i in set03:
    cntr = text.count(i)
    if cntr > len(i) and text.count(' ' + i.lower()) < (cntr - 2) and text.count('\n' + i.lower()) < (cntr - 2):
        set04.append(i)

In [194]:
print(len(set03), len(set04))

6531 756


There are many ways to do better with the names, but for now I have a list.

In [719]:
#set03

In [195]:
name = set04

Exceptions are now found character by character. After flipping by all the above by the rules, now we find every remaining capital letter. 
Algorithm : 
 - create a boolean array of the length of the character array, filled with False
 - everytime the rules and lists would trigger capitalization, turn the position on the array to True (don't touch the old array)
 - create a new string with a capitalization flip at every position marked as True
 - create a list of integers
 - iterate through the new string, and add the index of the character to integer list if it is a capital
 - from the last to first integer, subtract the index of the previous integer + 1 from the given index
 - (check that the calculations are correct)
 - calculate the bit cost to encode to encode the integer array for various types of encoding (gamma, delta, Fibinachi, triad, dynamic Hoffman?, omega?)

In [835]:
#bytesArr = bytearray(b'\x00\x00\x16\xa3') 

In [340]:
cflip = [False]*len(text) # marks if need to flip the letter by a rule

In [342]:
i = text[0:-1].find('.')
j=0
k=-1
while i >= 0:
    # j is set to i, the last found location of a punctuation mark
    j = i
    # find the next letter after the punctuation mark
    x = re.search('[A-Za-z]', text[j:-1])
    # end of text, no more letters
    if x == None:
        break
    # k is the position of the letter to capitalize
    k = x.start()+j
    # switch case of the letter
    if not(any(text[k-1] == item for item in num)):
        cflip[k] = True
    # find next instance of any type of punctuation or a new paragraph
    i2 = k+1+text[k+1:-1].find('\n\n')
    cont = True
    i1 = k+1
    while (cont):
        x = re.search('[.?!]', text[i1:-1])
        cont = False
        if x == None:
            #we have no more punctuation, only '\n\n' relevent (but will be -1)
            i1 = i2
        else:
            #set our index
            i1 = i1+x.start()
            # we need to now add an additional check for '.. '
            if (text[i1+1]=='.' and text[i1+2]==' '):
                i1 = i1 + 2
                #print(k, i1, text[i1-3:i1+2])
                cont = True                
    if i2 < k+1:
        #we have no more new paragraphs, only punctuation relevant
        #note that if x = None as well, this does nothing, and they both indicate that there are no more instances
        i2 = i1
    i = min(i1,i2)
    # no more punctuation and no more new paragraphs
    if i < k+1:
        break

In [343]:
# code is still taking a while but much faster
# titles I am skipping, so the next is for the letter I in isolation

for i in range(len(text)):
    if text[i] == 'I' or text[i] == 'i':
        if i == 0:
            if any(text[1] == item for item in sim):
                cflip[i] = True
        elif i == len(text) - 1:
            if any(text[i-1] == item for item in sim):
                cflip[i] = True
        else:
            if any(text[i-1] == item for item in sim) and any(text[i+1] == item for item in sim):
                cflip[i] = True

In [ ]:
# much faster
#for i in range(10000):
#    if cflip[i]:
#        print(text[i-5:i+1])
# abr, num, sim, low, upp, cflip, text, name

In [ ]:
# now to flip for the abr list
# k, kk are the index of interest
for i in abr:
    k = 0      # reset the index
    l = len(i)
    for j in range(text.lower().count(i.lower())):
        k = k + 1 #k+l+1, but need to clear these from name
        kk = text[k:].lower().find(i.lower()) + k
        k = kk # set the index
        # if both sides are symbols 
        #if j == 0:
        if any(text[k-1] == item for item in sim) and any(text[k+l] == item for item in sim):
            for ii in range(l):
                cflip[k+ii] = True
            #print(i, cflip[k-1:k+l+1])

In [ ]:
# now to flip for the name list
# k, kk are the index of interest
for i in name:
    k = 0      # reset the index
    l = len(i)
    for j in range(text.lower().count(i.lower())):
        k = k + 1 #k+l+1, but need to clear these from name
        kk = text[k:].lower().find(i.lower()) + k
        k = kk # set the index
        # if both sides are symbols 
        if any(text[k-1] == item for item in sim) and any(text[k+l] == item for item in sim):
            cflip[k] = True
            #print(i, cflip[k-1:k+l+1])

In [1010]:
#print(text[1000:2000])

In [ ]:
tf = ""
for i in range(len(text)):
    if cflip[i]:
        tf += text[i].swapcase()
    else:
        tf += text[i]

In [75]:
# one instance of the really slow thing
#with open("text_flip_case.txt", "w") as f:
#    f.write(tf)

The name list appears to not be aggressive enough, but otherwise the file looks good.
Now we can create an exception list.

In [ ]:
exc = []
for i in range(len(tf)):
    if any(tf[i] == item for item in upp):
        exc.append(i)

In [ ]:
print(len(exc))

In [242]:
#note that encoding starts at 1, not 0, so should be shifted by one
#reverse order ti subtract the old index, not the new one
for i in range(len(exc) - 1):
    j = len(exc) - i - 1
    exc[j] = exc[j] - exc[j-1] # will be 1 if the numbers are next to each other, but encodings start at 1
exc[0] = exc[0] + 1 # to start at 1, not 0

In [ ]:
# check that exc now points to the correct exceptions
# looks good

In [244]:
print(len(exc), max(exc))

17850 4359


Now we need to figure out encoding lengths.

Ternary code wins. It will do even better if I am more agressive with the names. But that is for later.

The current total count is 277 + 3 + 12610 + 3 + 237225 = 250118 bits = 31265 bytes. Note that the +3 between them are an additional space after each of the character strings -- 1 space = end of element, 2 spaces = end of list. The one flaw with this is that I don't have an end of file character for the ternary code, so it needs to be added to the end of the encoded text, not the beginning. I could also define the maximum number first, then give the array, and a number greater than the maximum ends the array.

Now I need the code to actually store this binary. I need to figure out how to do that in Python 3+. I believe I also need to figure out how to fill in extra space for the two additional bits which are not included yet. 

However, lets recalculate byte length with the requirement of a whole number of bytes.

In [ ]:
(295+3)/8 # all caps lists

In [791]:
(23653 + 3+3)/8 # 3 extra here for a whole number of bytes -- names list

2957.375

In [793]:
171410/8 # and now 1 byte over; however, I can add whatever if I go beyond the length of the text.

21426.25

In [232]:
(237225+15)/8 #however, now it is a whole number

29655.0

In [1124]:
#total count
(277+3+12610 + 3+3+237225+15)/8
#two more than the previous estimate

31267.0

The rest of this is to program the encoding. I will deal with it later if I have time.

abr, name, exc #

In [246]:
# - store bits and bytes, take bits and add it to the byte list (input array of bools, update encoding)
class byte_mng:
    def __init__(self):
        self.byte_store = [0] # here I am filling the bytes, should be integers from 0 to 255
        self.bit_count = 0    # this is the number of bits in byte_store which are full; ensures that I don't overfill bytes and send them to the string when ready
        self.j = 0            # counts the length of byte_store
        
    def load_bits(self, bits, lngth):
        # the old bits are already shifted appropriately
        jj = self.byte_store[self.j]
        
        # bit shift new bits
        if self.bit_count < (8 - lngth): 
            ii = bits << int(8 - lngth - self.bit_count)
        elif self.bit_count == (8 - lngth):
            ii = bits
        else:
            ii = bits >> (self.bit_count - 8 + lngth) # The excess will go off the end. I will encode them later.

        #print(lngth, self.bit_count, jj, ii)
        # bitwise or is the same as + in this case
        self.byte_store[self.j] = jj | ii
        used_bits = 8 - self.bit_count     # for when bit_count + lngth > 7
        self.bit_count += lngth

        bits_n = bits
        lngth_n = lngth

        #print(lngth, self.bit_count, jj, ii)
        # now, to work on the next byte
        while self.bit_count > 7:
            #print('here')
            self.j += 1
            self.bit_count -=8
            
            # we now need to take only the right most bit_count bits and put it in ii
            lngth_n -= used_bits
            bits_n = bits_n % (2**lngth_n) # removes the used bits
            if self.bit_count < 8:
                ii = (bits_n << 8 - lngth_n)%256
            else:
                ii = bits_n >> (lngth_n - 8)
                used_bits = 8
                
            self.byte_store.append(ii)

    # load letter and load word are for the abreviation list and name list
    def load_letter(self, cha):
        #000xx = stop
        #0010x = stop
        #0011x = ab
        #01xxx = c-j
        #10xxx = k-r
        #11xxx = s-z

        code_a = int('110', 2)
        num = ord(cha) - ord('a') + code_a # the order is unchanged, but the numbering starts earlier
        if num > 0 and num < 32:
            self.load_bits(num, 5)
        elif num <= 0:
            self.load_bits(0,3) # stop condition
        else:
            self.load_bits(2,4) # if I need another indicator

    # this will load the whole word into the bit string, then add a stop condition
    def load_word(self, word):
        word2 = word.lower()
        word2 = word2 + ' '
        for i in range(len(word2)):
            self.load_letter(word2[i])

    # now for the code for ternary code
    def load_ternary(self, num):
        # 00, 01, 10, 11
        # leading will be 1 or 0

        # number of digits for base 3 representation
        n = 0
        x = num
        while x >= 1:
            x = x / 3
            n += 1
        n2 = n

        # start with the leading bit
        # there are two variants only: 1 or 2
        n = n-1
        x = num // 3**n
        y = num % 3**n

        #print(x,y, x-1)
        self.load_bits(x-1,1) # 1 for 2 and 0 for 1
        
        # repeat for the rest of the digits
        for i in range(n2-1):
            n = n-1
            x = y // 3**n
            y = y % 3**n
            #print(x,y,1+x)
            self.load_bits(1+x,2) # 01 = 0, 10 = 1, 11 = 2

        # and load the end of number code
        self.load_bits(0,2)
            
    def save_bits(self, filename):
        self.byte_encode = bytearray(self.byte_store)
        with open(filename, "wb") as f:
            f.write(self.byte_encode)

In [248]:
print(exc[0:50])

[1, 5, 5, 6, 3, 6, 3, 6, 13, 6, 9, 7, 8, 7, 13, 18, 1, 4, 1, 3, 9, 4, 9, 11, 1, 4, 1, 3, 12, 15, 9, 22, 8, 19, 15, 10, 15, 35, 10, 25, 30, 8, 11, 6, 9, 5, 9, 11, 16, 15]


In [252]:
for i in range(len(abr)):
    abr[i] = abr[i].lower()
for i in range(len(name)):
    name[i] = name[i].lower()

In [254]:
cap_bits = byte_mng()

for i in abr:
    cap_bits.load_word(i)
cap_bits.load_letter(' ') # double space means end of list
for i in name:
    cap_bits.load_word(i)
cap_bits.load_letter(' ') # double space means end of list 
cap_bits.load_ternary(len(exc)) # size of the exc array, so that I can just append the next part of the encryption
# after all of the numbers, the rest of the byte is filled with zeros. the next segment begins at the next bit
for i in exc:
    cap_bits.load_ternary(i)

#cap_bits.save_bits('test_capital')

In [256]:
print(cap_bits.byte_store[-1], cap_bits.bit_count) # I still officially need to remove the last one if it exists, but it is fine here

184 7


Something here is saved. But I need to decode it to check that it is correct. The size is correct, however.

# encryption of the uncapitalized text

In [ ]:
# lets first reload everything needed, so we can work from here

In [258]:
#libraries
import string
import re
import numpy as np

#load text
f = open("MrZoat_WithThisRingS01_Gould.txt")
text = f.read()
f.close()
#print(text[0:200])

text = text.lower()


# these are most likely not the most effective variants, but it is irrelevant here
sim = ['*', '+', '|', '/', ':', '-', ';', '\'', '\"', ',', '.', '?', '!', '$', '%', '&', '}', '[', ']', '(', ')', '<', '>', ' ', '\n']

low = []
for i in range(26):
    low.append(string.ascii_lowercase[i])
num = []
for i in range(10):
    num.append(string.digits[i])

In [260]:
# now in the correct order
sim1 = ['\n', ' ', '!', '\"', '$', '%', '&', '\'', '(', ')', '*', '+', ',', '-', '.', '/' ]
sim2 = [':', ';', '<', '>', '?', '[', ']']
sim3 = ['|', '}']
ch = sim1 + num + sim2 + low + sim3 # this is our full list of symbols
len(ch)

61

In [12]:
#for i in range(61):
#    print(ord(ch[i]))

In [14]:
#print(len(ch), len(ch)**3)

In [16]:
#np.shape(counts)

In [50]:
# numpy to make 3-D arrays easily; they are all counts
count3 = np.zeros((61,61,61), int)

str3 = '000'

# again, this is very slow
# I don't think predefining str3 helps
for i in range(61):
    for j in range(61):
        for k in range(61):
            str3 = ch[i]+ch[j]+ch[k]
            count3[i,j,k] = text.count(str3)
            
num_non0 = np.count_nonzero(count3) # much smaller -- 94.4% of cases don't exist
print(num_non0)
c3_non0 = np.nonzero(count3)

12699


In [51]:
# initialize variables
ii = jj = kk = 0
str4 = '0000'
count4 = np.zeros((num_non0, len(ch)), int)

for i in range(num_non0):
    ii=c3_non0[0][i]
    jj=c3_non0[1][i]
    kk=c3_non0[2][i]
    for j in range(len(ch)):
        str4 = ch[ii]+ch[jj]+ch[kk]+ch[j]
        count4[i,j] = text.count(str4)

num4_non0 = np.count_nonzero(count4)
print(num4_non0)
c4_non0 = np.nonzero(count4)

59088


In [52]:
str2 = '00'
count2=np.zeros((61,61), int)

for i in range(61):
    for j in range(61):
        str2 = ch[i]+ch[j]
        count2[i,j] = text.count(str2)

num2_non0 = np.count_nonzero(count2)
print(num2_non0)
c2_non0 = np.nonzero(count2)

1427


In [53]:
# initialize variables
ii = jj = 0
str3 = '000'
count3n = np.zeros((num2_non0, len(ch)), int)

for i in range(num2_non0):
    ii=c2_non0[0][i]
    jj=c2_non0[1][i]
    for j in range(len(ch)):
        str3 = ch[ii]+ch[jj]+ch[j]
        count3n[i,j] = text.count(str3)

num3n_non0 = np.count_nonzero(count3n)
print(num3n_non0)
c3n_non0 = np.nonzero(count3n)

12699


In [54]:
count1=np.zeros((61), int)

for i in range(61):
    count1[i] = text.count(ch[i])

num1_non0 = np.count_nonzero(count1)
print(num1_non0)
c1_non0 = np.nonzero(count1)

61


In [ ]:
#np.count_nonzero(count4 > 1)

print(61*61, np.count_nonzero(count2), np.count_nonzero(count2 > 1))

#3721 1427 1258

The greatest number of states we can have is 10158 + 1258 + 61 = 11477. I assume that the next step is to program the dynamic PPM model. This uses the dynamic arithmetic model. The recommended rate of new is half the number of symbols. 

We actually need to add a 4th character onto this for the calculation. 

If we try to store the contextual version for every step of the dynamic process, we will need to for every letter save the machine. For the first step, we can use 1 bit, but there are 4 million characters, so 4 million statistics which we need to save. For the second step, we need to save 2 numbers and 1 character, and the size keeps growing from there. Saving all of this data is not possible. 

We can instead ask what it costs to save all the contexts for our 42210 + 10158 + 1258 + 57 + 1 = 53684 potential counts. We can also calculate for 59088 + 12699 + 1427 + 61 + 1 = 73276 when we just want to calculate for non-empty counts. I am not quite sure when we should cut off a context.

Each character can be encoded in 6 bits as we have 61 characters and counts can be given with a number encoding. Characters can be further reduced by examining spacing between characters rather than needing to provide the characters, as the structure is ordered.

 - Provide array of non-zero symbols. (25 bytes typically, although this could be reduced by counting spacing instead.) If we encode each character in 8 bits, we may not need to provide this. We end this with a border (like 'A' or a number which takes the spacing beyond ASCII).
 - Provide counts for context '': Each count is 1 character and one positive integer long. We have 61 counts.
 - Provide counts for context 'x': Each count is 2 characters and one positive integer long. We have 1427 counts.
 - Provide counts for context 'xx': Each count is 3 characters and one positive integer long. We have 12699 counts.
 - Provide counts for context 'xxx': Each count is 4 characters and one positive integer long. 
 - End of each array can be indicated by:
   a) With full character encoding, or 6 bit encoding, the next context will start again at the begining of the alphabet.
   b) With only spacing counted, we can save a mark or just go beyond the end of our character array length.

Each character is 1 byte. Our maximum count is 724362, which requires 20 bits to encode. We could do this with 3 bytes (but not 2 bytes), but 4 bytes is more typical. Total cost, with typical encoding:
5* 61 + 6 * 1427 + 7 * 12699 + 8 * 59088 = 570464 bytes = 0.57 MB.

Spacings: (the last 3 is for the end of the list)
This assumes all digits and letters are used.
 - p1: {10, 22, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 2, 2, 5, 1, 3} -- 3 at the end is used to indicate end of file
 - p2: {count1} -- without any charater spacing, because all are used
 - p3: {c2_non0\[1\]\[0\]+1, c2_non0\[1\]\[1:\]- c2_non0\[1\]\[0:-1\]} {count2} -- The first character spacing is removed, since all are going to exist. The second is as given, but the negative values in it need to be recalculated (+61 I think, I need to check this). This will indicate incrementing the first character by 1. Going to 61 is the most effecient version, and works. The additional +1 is already included.
 - x = 60 - the last index -- this will encode the first non-existing count, which indicates end of list
 - p4: {c3_non0\[1\]\[0\]+1, c3_non0\[1\]\[1:\]- c3_non0\[1\]\[0:-1\]} {c3_non0\[2\]\[0\]+1, c3_non0\[2\]\[1:\]- c3_non0\[2\]\[0:-1\]} {count3} -- Rules like the previous.
 - y = 60 - the last index -- as x
 - p5: {c4_non0\[1\]\[0\]+1, c4_non0\[1\]\[1:\]- c4_non0\[1\]\[0:-1\]} {c4_non0\[2\]\[0\]+1, c4_non0\[2\]\[1:\]- c4_non0\[2\]\[0:-1\]} {c4_non0\[3\]\[0\]+1, c4_non0\[3\]\[1:\]- c4_non0\[3\]\[0:-1\]} {count4} -- For this case, the formalism is not actually correct for how I have calculated the data, which uses c3 numbers for calculating c4 numbers. However, the structure for transmition is similar
 -  z = 60 - the last index, to indicate end of file, may now add the text

All numbers will be encoded with ternary code, to maintain consistency, despite not being the best choise. I should calculate costs, separating spacings and counts as two separate encodings. Counts most likely want ternary code, while this is likely inefficent for spacings.

9 bits is the greatest spacing. However, this encoding doesn't quite work. The alternative encoding, which I did for c4, does work. 
 - p1: {10, 22, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 2, 2, 5, 1, 3}
 - p2: {count1}
 - p3: {c2_non0\[1\]\[0\]+1, c2_non0\[1\]\[1:\]- c2_non0\[1\]\[0:-1\]} {count2}
 - x = 60 - the last index
 - p4: {c3n_non0\[1\]\[0\]+1, c3n_non0\[1\]\[1:\]- c3n_non0\[1\]\[0:-1\]} {count3n} -- Like p2, we have a context (the set of all non-zero contexts from p3), which always increments by 1, and our added letter counts. Therefore, everytime there will be exactly 2 numbers, a spacing and a count. We know which previous contexts exist from the previous process. We deal with negatives in our array like indicated before.
 - y = 60 - the last index
 - p5: {c4_non0\[1\]\[0\]+1, c4_non0\[1\]\[1:\]- c4_non0\[1\]\[0:-1\]} {count4} -- Like previous. This is the correct formulation of my calculation of counts.
 - z = 60 - the last index, to indicate end of file

In [158]:
#print(c2_non0[1][-5:])

[54 57 58  0 26]


In [160]:
#max(c2_non0[1][1:]- c2_non0[1][0:-1])

35

In [ ]:
#The second to last should be 3: 58+3 = 61 = loop back to 0

Except for the first few characters, every character has a 3 character preceding context. However, I wish to keep all the single count elements and the smaller machines due to how they count the context. I can recalculate the contexts as below:

 - p1: {10, 22, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 2, 2, 5, 1, 3}
 - p2: {'ABC'} -- The first three characters of the document, encoded from 1 - 61 (index 0 - 60). 
 - p3: {c2_non0\[1\]\[0\]+1, c2_non0\[1\]\[1:\]- c2_non0\[1\]\[0:-1\]} -- Add 61 to all non-positive.
 - x = 60 - the last index (note, last index here, not last spacing)
 - p4: {c3n_non0\[1\]\[0\]+1, c3n_non0\[1\]\[1:\]- c3n_non0\[1\]\[0:-1\]} -- Add 61 to all non-positive.
 - y = 60 - the last index (note, last index here, not last spacing)
 - p5: {c4_non0\[1\]\[0\]+1, c4_non0\[1\]\[1:\]- c4_non0\[1\]\[0:-1\]} {count4} -- Add 61 to all non-positive character spacings. Counts are only important here, as the old counts can be calculated from these counts and the first 3 characters.
 - z = 60 - the last index, to indicate end of file (note, last index here, not last spacing)

I should be able to here calculate the bit / byte length and figure out which number encodings are better. 

***

I have a problem with the encoding -- if the index of the last element of the old context is less than the first element of the next, the encoder / decoder will not recognize the requirement to increment to the next element. I can fix this by inserting an unused number (62 or 63) between the two elements, to indicate the need to increment. I have not yet added this fix to the algorithm (due to time constraints). The increase in size should be relatively small. 

Also, I can just put the indicies in straight, and not use the spacings. This doesn't alter the space usage, but decreases processing time. 

The first array needs to have a separate encoding (and be based on spacing) as we don't know the size of our array until after reading the first array. (bit size -> 7, 6, 6, 6, {5}, 5, ..., 5, {4}, 4, 4, 4, 4, {2}, 2 or just all 7) Now we can calculate the size of the array = 26 + 10 + number of special elements. We have 1 more element as our stop, to indicate needing to increment the first index by one if it is not obvious. When this index reaches the size of our previous array, we know that the next number will be for the next array. The last byte is finished with zeros (they will be ignored).

All future calculation of efficency is not effected by this change. The size will increase, but not significantly. 

Our count number also needs something to indicate the size of the array. After the p5 array, before the extra 0s and count array, I should add a ternary encoded number for the number of bits in the maximum count, defining the size of the next array. I don't want a fixed-length encoding here, in order to not assume the size of the maximum count. Then I should ignore the trailing 0s until the end and load the next numbers straight onto the byte string, as there is no reason to believe they will be a whole number of bits (although they are).

In [291]:
#sum([2, 1, 2, 2, 5, 1, 3])

16

In [262]:
np.max(count4) # This number is in my table. Lets bring the table here.

42912

In [ ]:
# calculate p3, p4, p5 character spacings, x,y,z

In [108]:
p1_arr = [10, 22, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 2, 2, 5, 1, 3]

Unencoded 6 bit length for the spacings is optimal, it looks like. I can just add zeros to the end to reach a full byte if needed due to the fact that I already signalled the end of file. It is likely unneeded.

In [268]:
#count4
c4_non0

(array([    0,     0,     0, ..., 12696, 12697, 12698], dtype=int64),
 array([ 2, 13, 33, ..., 58, 43,  0], dtype=int64))

In [266]:
c4_arr = [0]*num4_non0
for i in range(num4_non0):
    c4_arr[i] = count4[c4_non0[0][i], c4_non0[1][i]]

Our total cost after encoding is 173 kB, which is less than half the old calculation. Note that, despite the unencoded form, there are still two separate types of encoding due to the difference in bit length of the numbers. I wonder if I can just label them as uint * 6 and uint * 16 and just write the binary. There is no issue with putting the counts at the end, as we already know the order and number of counts, so we know what to expect. We need 4 extra zeros to reach a whole number of bits, but they still can be added to the end of the file.

170 kB, success, although I need to read it properly to truely check. Also .txt is not correct.

In [270]:
c4_arr[0:20]

[4, 1, 4, 1, 3, 4, 3, 8, 5, 6, 18, 3, 24, 17, 1, 7, 6, 8, 8, 2]

In [272]:
c4_arr[-20:]

[1, 4, 1, 1, 2, 1, 4, 7, 1, 110, 8, 2, 2, 6, 1, 1, 1, 2, 1, 1]

New variant:

 - p1: {10, 22, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 2, 2, 5, 1, 3}
 - size: (bit size -> 7, 6, 6, 6, {5}, 5, ..., 5, {4}, 4, 4, 4, 4, {2}, 2 or just all 7)
 - p2: {'ABC'} -- The first three characters of the document (index 0 - 60). 
 - p3: {c2_non0\[1\]\[0\]+1, c2_non0\[1\]\[1:\]- c2_non0\[1\]\[0:-1\]} -- Add 61 to all non-positive.
 - x = 61
 - p4: {c3n_non0\[1\]\[0\]+1, c3n_non0\[1\]\[1:\]- c3n_non0\[1\]\[0:-1\]} -- Add 61 to all non-positive.
 - y = 61
 - p5: {c4_non0\[1\]\[0\]+1, c4_non0\[1\]\[1:\]- c4_non0\[1\]\[0:-1\]} -- Add 61 to all non-positive character spacings. Counts are only important here, as the old counts can be calculated from these counts and the first 3 characters.
 - z = 61, to indicate end of file 
 - w = (bin(max(count4)) - 2) = number of bits in max count - ternary encoding
 - count4

Indicies encoded 0-60 (straight) 61 indicates need to increment the first index only if not obvious.

In [323]:
# - store bits and bytes, take bits and add it to the byte list (input array of bools, update encoding)
class byte_mng:
    def __init__(self):
        self.byte_store = [0] # here I am filling the bytes, should be integers from 0 to 255
        self.bit_count = 0    # this is the number of bits in byte_store which are full; ensures that I don't overfill bytes and send them to the string when ready
        self.j = 0            # counts the length of byte_store
        
    def load_bits(self, bits, lngth):
        # the old bits are already shifted appropriately
        jj = self.byte_store[self.j]
        
        # bit shift new bits
        if self.bit_count < (8 - lngth): 
            ii = bits << (8 - lngth - self.bit_count)
        elif self.bit_count == (8 - lngth):
            ii = bits
        else:
            ii = bits >> (self.bit_count - 8 + lngth) # The excess will go off the end. I will encode them later.

        #print(lngth, self.bit_count, jj, ii)
        # bitwise or is the same as + in this case
        self.byte_store[self.j] = jj | ii
        used_bits = 8 - self.bit_count     # for when bit_count + lngth > 7
        self.bit_count += lngth

        bits_n = bits
        lngth_n = lngth

        #print(lngth, self.bit_count, jj, ii)
        # now, to work on the next byte
        while self.bit_count > 7:
            #print('here')
            self.j += 1
            self.bit_count -=8
            
            # we now need to take only the right most bit_count bits and put it in ii
            lngth_n -= used_bits
            bits_n = bits_n % (2**lngth_n) # removes the used bits
            if self.bit_count < 8:
                ii = (bits_n << 8 - lngth_n)%256
            else:
                ii = bits_n >> (lngth_n - 8)
                used_bits = 8
                
            self.byte_store.append(ii)

    # now for the code for ternary code
    def load_ternary(self, num):
        # 00, 01, 10, 11
        # leading will be 1 or 0

        # number of digits for base 3 representation
        n = 0
        x = num
        while x >= 1:
            x = x / 3
            n += 1
        n2 = n

        # start with the leading bit
        # there are two variants only: 1 or 2
        n = n-1
        x = num // 3**n
        y = num % 3**n

        #print(x,y, x-1)
        self.load_bits(x-1,1) # 1 for 2 and 0 for 1
        
        # repeat for the rest of the digits
        for i in range(n2-1):
            n = n-1
            x = y // 3**n
            y = y % 3**n
            #print(x,y,1+x)
            self.load_bits(1+x,2) # 01 = 0, 10 = 1, 11 = 2

        # and load the end of number code
        self.load_bits(0,2)
            
    def save_bits(self, filename):
        self.byte_encode = bytearray(self.byte_store)
        with open(filename, "wb") as f:
            f.write(self.byte_encode)

In [325]:
def np2D_to_list1D(arr, mng_arr, m_ind = 0):
    out_arr = [arr[0]]
    max_ind = max(max(arr) + 1, m_ind) # so that giving this value is also allowed

    for i in range(1,len(arr)):
        # this is my exception, when the increase is not obvious
        # first condition -- that the old array increments, 
        # second condition, that the new index is greater than the previous, which won't increment the first normally
        if (mng_arr[i] != mng_arr[i-1]) and (arr[i] > arr[i-1]):
            out_arr.append(max_ind)
        out_arr.append(arr[i])

        # small bit of error-checking
        if (mng_arr[i] != mng_arr[i-1]) and (mng_arr[i] != (mng_arr[i-1] + 1)):
            print('error: ', i, ', ', mng_arr[i], ', ', mng_arr[i-1])

    return out_arr

In [327]:
counts_enc = byte_mng()

# p1
# 0 means ending the array
sim_arr = [10, 22, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 2, 2, 5, 1, 0]
# - size: (bit size -> 7, 6, 6, 6, {5}, 5, ..., 5, {4}, 4, 4, 4, 4, {2}, 2 or just all 7)
# should be calculated automatically:  calc_sim_size(sim)
size_sim_arr = [7, 6, 6, 6, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 4, 4, 4, 4, 4, 2, 2]

for i in range(len(sim_arr)):
    counts_enc.load_bits(sim_arr[i], size_sim_arr[i])

# p2: {'ABC'} -- The first three characters of the document (index 0 - 60). 
f3_num = [ch.index(text[0]), ch.index(text[1]), ch.index(text[2])]
for i in f3_num:
    counts_enc.load_bits(i, 6)

print('p2: ', len(f3_num), len(counts_enc.byte_store))

# p3_arr = c2_non0[1][:], add 61 for unclear boundaries
p3_arr = np2D_to_list1D(c2_non0[1][:], c2_non0[0][:], 61)
for i in p3_arr:
    counts_enc.load_bits(i, 6)

counts_enc.load_bits(61, 6)

print('p3: ', len(p3_arr), len(counts_enc.byte_store))

# p4_arr = c3n_non0[1][:], add 61 for unclear boundaries
p4_arr = np2D_to_list1D(c3n_non0[1][:], c3n_non0[0][:], 61)
for i in p4_arr:
    counts_enc.load_bits(i, 6)

counts_enc.load_bits(61, 6)

print('p4: ', len(p4_arr), len(counts_enc.byte_store))

# p5_arr = c4_non0[1][:], add 61 for unclear boundaries
p5_arr = np2D_to_list1D(c4_non0[1][:], c4_non0[0][:], 61)
for i in p5_arr:
    counts_enc.load_bits(i, 6)

counts_enc.load_bits(61, 6)

print('p5: ', len(p5_arr), len(counts_enc.byte_store))

# - w = (bin(max(count4)) - 2) = number of bits in max count - ternary encoding
w = len(bin(np.max(count4))) - 2 # 2 characters are for the indication of being a binary number
counts_enc.load_ternary(w)

print('w = ', w) # I believe this is 16 here

# - count4
for i in c4_arr:
    counts_enc.load_bits(i, w)

print('p6: ', len(c4_arr), len(counts_enc.byte_store))

# remove the trailing null byte if it exists

#counts_enc.save_bits('count_enc_test')

p2:  3 18
p3:  1429 1091
p4:  12848 10728
p5:  61366 56753
w =  16
p6:  59088 174930


In [329]:
print(counts_enc.byte_store[-1], counts_enc.bit_count) # also fine; but need to add functionality in future

128 1


# Pandas
lets save variables to compare fully and to use for error-checking

Dynamic version begins below. I need to update for the static version defined above. 
Now to encode the text. 

I need a bytearray which stores the data. I could just define an integer, however. In Python, integers have not limit to their length, so I can just push << numbers and shove on more bytes indefininitely, and use binary or to add new numbers to the end (a | b). This is really easy to add onto, while the bytearray requires being added in bytes. I can, however, store the numbers in a separate place (like an integer) and append them byte per byte when ready.

# Text Compression

My assumption is that I already have the capitalization, which I will not deal with here, and the context-based probabilities, which are already loaded. Do to how I store the probabilities, I will ignore the first three characters (already recorded in the model) and work only with probabilities based on the previous three characters.

I need routines to: 
{l_old, h_old, bits_old -> a, b -> h-l+1, h-l+1 / 2^n -> a+ term, b+ term -> l, h -> bin l, bin h -> enc update -> new bin l, new bin h, bits_new -> new l, new h}
 - find the counts based on the context (give context, find counts)
 - find the probabilities and borders based on the counts (give counts and char, find left and right border)
 - store bits and bytes, take bits and add it to the byte list (input array of bools, update encoding)
 - keep track of and update l, h, bits

I can fix the end of file issue by adding 3 bits to the beginning, setting them to 0, then later setting them to encode how many zeros at the end of the text are not intended / don't indicate extra cycles. This prevents from needing to play around with the order of the files.

In [282]:
# ch                             -- (character indicies)
# count4 [num3_non0, len(ch)]    -- (our counts)
# c3_non0[:3][:len(num3_non0)]   -- (to find the first index based on character indicies)

# first 3 characters
# f3ch (subtract 1 from each of the three)
# context = [f3ch[0]-1, f3ch[1]-1, f3ch[2]-1]
# push context each time (new = {0,1,2}) ... 

# counts = count4, context = 'XYZ', char = 'W'
#def find_count(counts, context, char):
#    i,j,k,m = find_indicies(context, char)
#    return counts(ii, jj)

# context = [x, y, z] char = w, where w,x,y,z are all indicies
# I am storing the context in the form of indicies, not characters
def find_count(counts, context, char):
    #ii = c3_non0[0][i] #<- find X first 'X' -> k =  ch.index('X')
    i1 = np.nonzero(c3_non0[0]==context[0])[0][0]
    if context[0] < 60:
        i2 = np.nonzero(c3_non0[0]==context[0]+1)[0][0] 
    else:
        i2 = len(c3_non0[0])
    j1 = np.nonzero(c3_non0[1][i1:i2]==context[1])[0][0] + i1
    j2 = 0
    j_store = np.nonzero(c3_non0[1][i1:i2]==context[1]+1)[0] #<- may be -1
    if len(j_store) > 0:
        j2 = np.nonzero(c3_non0[1][i1:i2]==context[1]+1)[0][0] + i1
    else:
        j2 = len(c3_non0[1])
    k = np.nonzero(c3_non0[2][j1:j2]==context[2])[0][0] + j1 #<- exactly one

    # check the borders here

    # all the counts to the correct one, the correct count, all the counts after
    return sum(count4[k, :char]), count4[k, char], sum(count4[k, char+1:])

In [315]:
# - find the probabilities and borders based on the counts (give counts, find left and right border)
import math

def find_probs(counts):
    den = float(counts[0]+counts[1]+counts[2])
    num1 = float(counts[0])
    num2 = float(counts[0]+counts[1])
    lb_fl = num1 / den
    rb_fl = num2 / den
    win = 2**40
    #print(lb_fl, rb_fl, win)
    lb = math.ceil(lb_fl * win)
    rb = math.ceil(rb_fl * win) - 1
    return lb, rb

In [286]:
# - store bits and bytes, take bits and add it to the byte list (input array of bools, update encoding)
class byte_mng:
    def __init__(self):
        self.byte_store = [0] # here I am filling the bytes, should be integers from 0 to 255
        self.bit_count = 0    # this is the number of bits in byte_store which are full; ensures that I don't overfill bytes and send them to the string when ready
        self.j = 0            # counts the length of byte_store
        
    def load_bits(self, bits, lngth):
        # the old bits are already shifted appropriately
        jj = self.byte_store[self.j]
        
        # bit shift new bits
        if self.bit_count < (8 - lngth): 
            ii = bits << (8 - lngth - self.bit_count)
        elif self.bit_count == (8 - lngth):
            ii = bits
        else:
            ii = bits >> (self.bit_count - 8 + lngth) # The excess will go off the end. I will encode them later.

        #print(lngth, self.bit_count, jj, ii)
        # bitwise or is the same as + in this case
        self.byte_store[self.j] = jj | ii
        used_bits = 8 - self.bit_count     # for when bit_count + lngth > 7
        self.bit_count += lngth

        bits_n = bits
        lngth_n = lngth

        #print(lngth, self.bit_count, jj, ii)
        # now, to work on the next byte
        while self.bit_count > 7:
            #print('here')
            self.j += 1
            self.bit_count -=8
            
            # we now need to take only the right most bit_count bits and put it in ii
            lngth_n -= used_bits
            bits_n = bits_n % (2**lngth_n) # removes the used bits
            if self.bit_count < 8:
                ii = (bits_n << 8 - lngth_n)%256
            else:
                ii = bits_n >> (lngth_n - 8)
                used_bits = 8
                
            self.byte_store.append(ii)
            
    def save_bits(self, filename):
        self.byte_encode = bytearray(self.byte_store)
        with open(filename, "wb") as f:
            f.write(self.byte_encode)

In [317]:
# - keep track of and update l, h, bits
# {l_old, h_old, bits_old -> a, b -> h-l+1, h-l+1 / 2^n -> a+ term, b+ term -> l, 
#  h -> bin l, bin h -> enc update -> new bin l, new bin h, bits_new -> new l, new h}
class arith_code_mng:
    def __init__(self, context):
        # window size
        self.N = 40
        self.win_size = 2**self.N

        # left and right borders
        # I know I don't need so many, but it should be fine right now
        self.l = 0
        self.h = self.win_size - 1
        self.bits = 0
        self.lo = 0
        self.ho = self.win_size - 1
        self.ln = 0
        self.hn = self.win_size - 1

        # context
        self.context = context
        self.char = 0

        # counts
        # this is currently global

        # encoding
        self.byte_encode = byte_mng()

        self.duration_static = 0

    def add_character(self, char):
        self.char = ch.index(char)
        counts = find_count(count4, self.context, self.char)
        a,b = find_probs(counts)

        #print("ch: ", char, "a,b: ", format(a, '040b'), format(b, '040b'))
        

        hl1_N = float(self.ho-self.lo+1) / float(self.win_size)
        a_pl = math.ceil(hl1_N * a)
        b_pl = math.ceil(hl1_N * (b + 1)) - 1

        #print("num: ", float(self.ho-self.lo))
        #print(format(self.window, '040b'))


        self.l = self.lo + a_pl
        self.h = self.lo + b_pl

        #print("ch: ", char, "l,h:", format(self.l, '032b'), format(self.h, '032b'))


        #print("new l, h: ", float(self.h-self.l))
        enc_n, bit_n = self.encode() # loads bits into the bit array and records how many bits need to be removed from l and h
        self.new_lh(enc_n, bit_n) # removes bits from l and h
        #print("remove: ", float(self.hn-self.ln), enc_n, bit_n)

        # updates for the next run
        self.lo = self.ln
        self.ho = self.hn
        self.update_context() # so that the next context uses the current character and removes the oldest

    # This appears to work properly.
    # This loads the bits into the array and updates bits, but so far doesn't update l and h
    def encode(self):
        l = self.l
        h = self.h

        nbitload = 0
        nbits = 0

        #print(self.h-self.l)
        # both l and h are N bits long
        x = self.win_size
        #print(l / x, h / x)

        lb = 0
        hb = 0
        
        for i in range(self.N):
            #print("bits:", i, lb,hb, l/x, h/x)
            x = x // 2
            lb = l // x
            l = l % x
            hb = h // x
            h = h % x
            #print("bits:", i, lb,hb, l/x, h/x, self.bits)
            if lb == hb:
                self.byte_encode.load_bits(lb, 1)
                while self.bits > 0:
                    self.byte_encode.load_bits(not(lb),1)
                    self.bits -= 1
                nbitload += 1
            else:
                #print(i)
                break

        # the final 0s in the file
        if nbitload == 0:
            self.duration_static += 1
        else:
            self.duration_static = 0

        # widening the window
        # print(l / x, h / x)
        for i in range(self.N - nbitload - 1):
            x = x // 2
            lb = l // x
            l = l % x
            hb = h // x
            h = h % x
            if lb == 1 and hb == 0:
                nbits += 1
                self.bits += 1
            else:
                break

        #print(self.h-self.l, nbitload, nbits)
        return nbitload,nbits

    # again, I need to check this and check the logic
    def new_lh(self, nbitload, nbits):
        # removing bits
        self.ln = (self.l << nbitload) % self.win_size
        self.hn = (self.h << nbitload) % self.win_size
        self.hn = self.hn | (2**nbitload - 1) # fill the end with 1s, not 0s

        # widening of the window
        if nbits > 0:
            l_fb = (self.ln // 2**(self.N-1)) << (self.N-1)
            h_fb = (self.hn // 2**(self.N-1)) << (self.N-1)
            self.ln = (self.ln % 2**(self.N-1-nbits)) << nbits
            self.hn = (self.hn % 2**(self.N-1-nbits)) << nbits
            self.hn = self.hn | (2**nbits - 1) # fill the end with 1s, not 0s
            # and return the first bit to the array
            self.ln = self.ln | l_fb
            self.hn = self.hn | h_fb

    def update_context(self):
        # This is not the most effective version of fofo list structure, but
        # I have only 3 elements, and it is the easiest to work with. 
        # I could get only 2 updates having a number which record the start, but
        # then I need to tell everything to access the array based on this.
        self.context[0] = self.context[1]
        self.context[1] = self.context[2]
        self.context[2] = self.char

    # I need error-checking here to prevent using any routines if this has already been applied
    def finalize(self):
        self.byte_encode.load_bits(1,1) # this is always 1, because 00, 01, 11 are the only possibilities, and 00 and 11 will already be removed.
        if self.duration_static > 0:
            self.byte_encode.load_bits(0, self.duration_static) # how many final characters the fraction was unchanged
            
        # now to finalize the bytestream
        a = 8 - self.byte_encode.bit_count
        # the encoding is already correct unless a = 8
        if a == 8:
            a = 0
            # remove the last zero byte
            b = self.byte_encode.byte_store.pop()
        return a

In [ ]:
#print(bytearray(encoder_new.byte_encode.byte_store))

In [321]:
context_start = [ch.index(text[0]), ch.index(text[1]), ch.index(text[2])]

encoder_new = arith_code_mng(context_start)

for i in text[3:]:
    encoder_new.add_character(i)

whitespace2 = encoder_new.finalize()
encoder_new.byte_encode.save_bits('text_encode') # This will be used to test the size of the compressed text once I run the algorithm. 
                                                 # I need to switch 'wb' to 'ab' for this to actually work as intended.
print(encoder_new.duration_static, whitespace2)

# I also need to add a number somewhere to indicate how many 0s are added to the end due to not being a full byte, and the size of the capitalization encoding,
# since neither this nor the captialization encoding have a definite end (I can add an end to the capitalization only when I know the text size)

0 2


In [319]:
whitespace2

0

In [ ]:
# link bits and write file
# I need to unify all my bit writters into one, although I also need know how many zeros to add, so perhaps I should have three whitespaces and 2 3-bit numbers indicating
# indicating the last two whitespace sizes. 

# First loader -- {whitespace1_size, whitespace2_size, length_of_capitalization, character_array_and_frequency_counts}
# Second loader -- {capitalization_locations, whitespace1}
# Third Loader -- {text_encripted, whitespace2}

# It is not the most effective, but I add at most 17 bits < 3 bytes, independent of the file size

# Linking Files

In [333]:
print(counts_enc.bit_count)

4


In [331]:
# add the whitespace to the end of counts
counts_enc.load_bits(whitespace2, 3)
counts_enc.save_bits('test_counts_2')

In [304]:
# remove the trailing null byte if it exists
# already done only for the arithmetic encoder
if cap_bits.bit_count == 0:
    cap_bits.byte_store.pop()
if counts_enc.bit_count == 0:
    counts_enc.byte_store.pop()

In [335]:
text_byte_array = bytearray(encoder_new.byte_encode.byte_store)
cap_byte_array = bytearray(cap_bits.byte_store)
count_byte_array = bytearray(counts_enc.byte_store)

In [337]:
filename = "full_text_encode_Gould"
with open(filename, "wb") as f:
    f.write(count_byte_array)
    f.write(cap_byte_array)
    f.write(text_byte_array)

# Old